In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# import data
data_folder = "ML_data"
holidays_events = pd.read_csv(os.path.join(data_folder, "holidays_events.csv"))
oil = pd.read_csv(os.path.join(data_folder, "oil.csv"))
sample_submission = pd.read_csv(os.path.join(data_folder, "sample_submission.csv"))
stores = pd.read_csv(os.path.join(data_folder, "stores.csv")) 
test = pd.read_csv(os.path.join(data_folder, "test.csv"))
train = pd.read_csv(os.path.join(data_folder, "train.csv"))
transactions = pd.read_csv(os.path.join(data_folder, "transactions.csv"))

In [3]:
train.shape

(3000888, 6)

In [4]:
# merge
train = train.merge(stores, on="store_nbr", how="left")
train = train.merge(transactions, on=["date", "store_nbr"], how="left")
train = train.merge(oil, on="date", how="left")

In [5]:
# change holoday df names before merging
# holidays_events = holidays_events.rename(columns=lambda x: f"holiday_{x}" if x != "date" else x)

In [6]:
train.shape

(3000888, 12)

In [7]:
train.head(3)

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN


### Per day per store

In [8]:
# transaction fill 
train["transactions"] = train["transactions"].fillna(0)

In [9]:
daily_store_sales = train.groupby(["date", "store_nbr", "city", "state", "type", "cluster","transactions"])["sales"].sum().reset_index()
daily_store_sales

,date,store_nbr,city,state,type,cluster,transactions,sales
0,2013-01-01,1,Quito,Pichincha,D,13,0.0,0.000000
1,2013-01-01,2,Quito,Pichincha,D,13,0.0,0.000000
2,2013-01-01,3,Quito,Pichincha,D,8,0.0,0.000000
3,2013-01-01,4,Quito,Pichincha,D,9,0.0,0.000000
4,2013-01-01,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,0.0,0.000000
...,...,...,...,...,...,...,...,...
90931,2017-08-15,50,Ambato,Tungurahua,A,14,2804.0,16879.121004
90932,2017-08-15,51,Guayaquil,Guayas,A,17,1573.0,20154.559000
90933,2017-08-15,52,Manta,Manabi,A,11,2255.0,18600.046000
90934,2017-08-15,53,Manta,Manabi,D,13,932.0,8208.189000


In [10]:
holidays=holidays_events[holidays_events['transferred']=='False']
holidays = holidays[(holidays['locale'] == 'National') | (holidays['locale'] == 'Local')]

In [11]:
# Add the 'is_national_holiday' column
holidays['is_national_holiday'] = holidays['locale'].apply(lambda x: 1 if x == 'National' else 0)

# Apply the conditional logic for 'locale_name'
holidays['locale_name'] = holidays.apply(
    lambda row: row['locale_name'] if row['is_national_holiday'] == 0 else None, axis=1
)

# Group by 'date' to get the highest value of 'is_national_holiday' (1 if any holiday on that date is national)
holidays_filtered = holidays.groupby('date').agg(
    is_national_holiday=('is_national_holiday', 'max'),
    locale_name=('locale_name', 'first')  # Keep 'locale_name' for reference
).reset_index()

# Select relevant columns and display the result
holiday = holidays_filtered[['date', 'is_national_holiday', 'locale_name']]

In [12]:
daily_store_sales=pd.merge(daily_store_sales,holiday,on='date',how='left')
daily_store_sales['is_local_holiday']=daily_store_sales['city']==daily_store_sales['locale_name']
daily_store_sales['is_holiday']=daily_store_sales['is_local_holiday'] | daily_store_sales['is_national_holiday']

In [13]:
daily_store_sales

,date,store_nbr,city,state,type,cluster,transactions,sales,is_national_holiday,locale_name,is_local_holiday,is_holiday
0,2013-01-01,1,Quito,Pichincha,D,13,0.0,0.000000,NaN,NaN,False,False
1,2013-01-01,2,Quito,Pichincha,D,13,0.0,0.000000,NaN,NaN,False,False
2,2013-01-01,3,Quito,Pichincha,D,8,0.0,0.000000,NaN,NaN,False,False
3,2013-01-01,4,Quito,Pichincha,D,9,0.0,0.000000,NaN,NaN,False,False
4,2013-01-01,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,0.0,0.000000,NaN,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
90931,2017-08-15,50,Ambato,Tungurahua,A,14,2804.0,16879.121004,NaN,NaN,False,False
90932,2017-08-15,51,Guayaquil,Guayas,A,17,1573.0,20154.559000,NaN,NaN,False,False
90933,2017-08-15,52,Manta,Manabi,A,11,2255.0,18600.046000,NaN,NaN,False,False
90934,2017-08-15,53,Manta,Manabi,D,13,932.0,8208.189000,NaN,NaN,False,False


In [14]:
# Drop unnecessary columns related to intermediate holiday flags
columns_to_drop = ["is_national_holiday", "locale_name", "is_local_holiday"]

# Drop the columns
daily_store_sales = daily_store_sales.drop(columns=columns_to_drop, errors='ignore')
daily_store_sales

,date,store_nbr,city,state,type,cluster,transactions,sales,is_holiday
0,2013-01-01,1,Quito,Pichincha,D,13,0.0,0.000000,False
1,2013-01-01,2,Quito,Pichincha,D,13,0.0,0.000000,False
2,2013-01-01,3,Quito,Pichincha,D,8,0.0,0.000000,False
3,2013-01-01,4,Quito,Pichincha,D,9,0.0,0.000000,False
4,2013-01-01,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4,0.0,0.000000,False
...,...,...,...,...,...,...,...,...,...
90931,2017-08-15,50,Ambato,Tungurahua,A,14,2804.0,16879.121004,False
90932,2017-08-15,51,Guayaquil,Guayas,A,17,1573.0,20154.559000,False
90933,2017-08-15,52,Manta,Manabi,A,11,2255.0,18600.046000,False
90934,2017-08-15,53,Manta,Manabi,D,13,932.0,8208.189000,False


In [15]:
# Export the cleaned dataset to a CSV file
output_file_path = "sales_per_day_per_store.csv"
daily_store_sales.to_csv(output_file_path, index=False)
